<b>Продвинутый SQL</b>

<b> Задача: </b>
Выполнить анализ базы данных пользователей stackoverflow с помощью SQL запросов <br>
<b> Описание данных<b>
* Таблица stackoverflow.badges Хранит информацию о значках, которые присуждаются за разные достижения. Например, пользователь, правильно ответивший на большое количество вопросов про PostgreSQL, может получить значок postgresql. 
* Таблица stackoverflow.posts Содержит информацию о постах.
* Таблица stackoverflow.users Содержит информацию о пользователях.
* Таблица stackoverflow.votes Содержит информацию о голосах за посты. 


In [ ]:
#!pip install sqlalchemy 
#!pip3 install psycopg2

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine 

Создадим конфигурацию для подключения к базе данных data-analyst-advanced-sql. 
Эта база данных содержит схему stackoverflow.

In [15]:
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': '*************', # пароль
    'host': '************',
    'port': 6432, # порт подключения
    'db': 'data-analyst-advanced-sql' # название базы данных
}  

In [16]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
) 
engine = create_engine(connection_string) 

# Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [27]:
query = '''
SELECT count(*)
FROM stackoverflow.posts
WHERE  post_type_id=1 and ( score>300 or favorites_count>=100)
'''

posts = pd.read_sql_query(query, con=engine)
print(posts['count'])

0    1355
Name: count, dtype: int64


# Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [28]:
query2 = '''
with i as(select creation_date::date, count(*)
from stackoverflow.posts 
where creation_date::date between '2008-11-01' and '2008-11-18' and post_type_id=1
group by creation_date::date)
select round(avg(count),0)
from i
'''
result2 = pd.read_sql_query(query2, con=engine)
print(result2)

   round
0  383.0


# Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [29]:
query3= '''
select count(distinct u.id)
from stackoverflow.badges b
full outer JOIN stackoverflow.users u on b.user_id=u.id
where u.creation_date::date=b.creation_date::date
limit 1
'''
result3 = pd.read_sql_query(query3, con=engine)
print(result3)

   count
0   7047


# Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [30]:
query4= '''
select count(distinct p.id)
from stackoverflow.posts p
join stackoverflow.votes v on p.id=post_id 
where p.user_id=3043 and v.creation_date is not null
'''

print(pd.read_sql_query(query4, con=engine))

   count
0     12


# Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [32]:
query5= '''
select *,
RANK()OVER (ORDER BY id desc)
from stackoverflow.vote_types
order by id
'''
display(pd.read_sql_query(query5, con=engine))

,id,name,rank
0,1,AcceptedByOriginator,15
1,2,UpMod,14
2,3,DownMod,13
3,4,Offensive,12
4,5,Favorite,11
5,6,Close,10
6,7,Reopen,9
7,8,BountyStart,8
8,9,BountyClose,7
9,10,Deletion,6


# Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [33]:
query6= '''
select u.id, count(vote_type_id)
from stackoverflow.users u 
join stackoverflow.votes v on u.id=v.user_id  
where v.vote_type_id=6
group by u.id
order by count desc, u.id desc
limit 10
'''
display(pd.read_sql_query(query6, con=engine))

,id,count
0,20646,36
1,14728,36
2,27163,29
3,41158,24
4,24820,23
5,9345,23
6,3241,23
7,44330,20
8,38426,19
9,19074,19


# Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [34]:
query7= '''
select u.id, count(distinct b.id),DENSE_RANK() OVER(ORDER BY count(b.id) desc) rnk_dense 
from stackoverflow.users u 
join stackoverflow.badges b on u.id=b.user_id  
where b.creation_date::date between '2008-11-15' and '2008-12-15'
group by u.id
order by count desc , u.id  
limit 10
'''
display(pd.read_sql_query(query7, con=engine))

,id,count,rnk_dense
0,22656,149,1
1,34509,45,2
2,1288,40,3
3,5190,31,4
4,13913,30,5
5,893,28,6
6,10661,28,6
7,33213,25,7
8,12950,23,8
9,25222,20,9


# Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
заголовок поста;
идентификатор пользователя;
число очков поста;
среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [35]:
query8= '''
select title,user_id,score,
round(avg(score) over (partition by user_id),0) as avg_score
from stackoverflow.posts
where title is not null and score!=0
'''
display(pd.read_sql_query(query8, con=engine))

,title,user_id,score,avg_score
0,Diagnosing Deadlocks in SQL Server 2005,1,82,573.0
1,How do I calculate someone's age in C#?,1,1743,573.0
2,Why doesn't IE7 copy <pre><code> blocks to the...,1,37,573.0
3,Calculate relative time in C#,1,1348,573.0
4,Wrapping StopWatch timing with a delegate or l...,1,92,573.0
...,...,...,...,...
47349,Multi-lingual projects in Visual Studio,3742716,1,1.0
47350,Change datatype when importing Excel file into...,3930756,5,5.0
47351,How to host a Silverlight app in a Sharepoint ...,4020932,8,8.0
47352,Getting counts for a paged SQL search stored p...,5696608,2,2.0


# Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [37]:
query9= '''
with i as (select user_id, count(id)
from stackoverflow.badges
group by user_id)

select title from stackoverflow.posts p
where p.title is not null and user_id=22656
'''
display(pd.read_sql_query(query9, con=engine))

,title
0,What's the strangest corner case you've seen i...
1,Project management to go with GitHub
2,What's the hardest or most misunderstood aspec...
3,What are the correct version numbers for C#?


# Напишите запрос, который выгрузит данные о пользователях . Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
* пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
* пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
* пользователям с числом просмотров меньше 100 — группу 3.<br>
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [66]:
query10= '''
select id, views,
      CASE
         WHEN views >= 350 THEN 1
         WHEN views >=100  and views < 350 THEN 2
         ELSE 3
      END as rank
from stackoverflow.users
where views>0
'''
display(pd.read_sql_query(query10, con=engine))

,id,views,rank
0,1,408587,1
1,2,23966,1
2,3,24396,1
3,4,73755,1
4,5,11700,1
...,...,...,...
23660,461895,10,3
23661,531489,2,3
23662,463160,5,3
23663,277558,18,3


# Дополните предыдущий запрос. 
Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. 
Выведите поля с идентификатором пользователя, группой и количеством просмотров. 
Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [67]:
query11= '''
WITH grp AS (SELECT g.id,
                    g.views,
                    g.group,
                    MAX(g.views) OVER (PARTITION BY g.group) AS max     
             FROM (SELECT id,
                          views,
                          CASE
                             WHEN views >= 350 THEN 1
                             WHEN views < 100 THEN 3
                             ELSE 2
                          END AS group
                   FROM stackoverflow.users
                   WHERE views > 0) as g
              )
  
SELECT grp.id, 
       grp.views,  
       grp.group
FROM grp
WHERE grp.views = grp.max
ORDER BY grp.views DESC, grp.id;
'''
display(pd.read_sql_query(query10, con=engine))

,id,views,rank
0,1,408587,1
1,2,23966,1
2,3,24396,1
3,4,73755,1
4,5,11700,1
...,...,...,...
23660,461895,10,3
23661,531489,2,3
23662,463160,5,3
23663,277558,18,3


# Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
* номер дня;
* число пользователей, зарегистрированных в этот день;
* сумму пользователей с накоплением.

In [43]:
query12= '''
with i as(select DATE_TRUNC('day', creation_date)::date as day,count(id) as users
from stackoverflow.users
where creation_date::date between '2008-11-01' and '2008-11-30'    
          group by DATE_TRUNC('day', creation_date)::date
     
         )


select EXTRACT('day' FROM day), users
,sum (users) over (order by day)  AS users_sum
from i
'''
display(pd.read_sql_query(query12, con=engine))

,date_part,users,users_sum
0,1.0,34,34.0
1,2.0,48,82.0
2,3.0,75,157.0
3,4.0,192,349.0
4,5.0,122,471.0
5,6.0,132,603.0
6,7.0,104,707.0
7,8.0,42,749.0
8,9.0,45,794.0
9,10.0,93,887.0


# Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
* идентификатор пользователя;
* разницу во времени между регистрацией и первым постом.

In [45]:
query13= '''
select u.id as user,
  min(p.creation_date)-min(u.creation_date) as delta

from stackoverflow.posts p
join stackoverflow.users u on p.user_id=u.id
where p.creation_date is not null
group by u.id 
'''
display(pd.read_sql_query(query13, con=engine))

,user,delta
0,30052,0 days 03:04:47
1,24416,0 days 00:21:57
2,4790,0 days 00:17:15
3,47051,2 days 03:02:01
4,37083,4 days 14:33:15
...,...,...
18844,49794,0 days 00:00:00
18845,34957,0 days 00:17:18
18846,4035,2 days 03:31:44
18847,4827,3 days 20:54:03


# Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [46]:
query14= '''
SELECT date_trunc('month', creation_date)::date as month,
       sum(views_count) as views
FROM stackoverflow.posts
GROUP BY month
ORDER BY views DESC
'''
display(pd.read_sql_query(query14, con=engine))

,month,views
0,2008-09-01,452928568
1,2008-10-01,365400138
2,2008-11-01,221759651
3,2008-12-01,197792841
4,2008-08-01,131367083
5,2008-07-01,669895


# Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [47]:
query15= '''
WITH  first_query as (
SELECT  p.creation_date dt_post,
        t.type,
        u.creation_date dt_user,
        p.user_id,
        display_name
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON u.id = p.user_id
JOIN stackoverflow.post_types t ON t.id = p.post_type_id
WHERE (DATE_TRUNC('day', p.creation_date) <= (DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month')) AND type = 'Answer'
),
final_data AS (
SELECT count(distinct(user_id)) AS sum_id,
       display_name,
       count(type) AS sum_answers
FROM  first_query
GROUP BY display_name
)

SELECT display_name,
       sum_id
FROM final_data
WHERE sum_answers > 100
ORDER BY display_name
'''
display(pd.read_sql_query(query15, con=engine))

,display_name,sum_id
0,1800 INFORMATION,1
1,Adam Bellaire,1
2,Adam Davis,1
3,Adam Liss,1
4,Alan,8
...,...,...
74,lomaxx,1
75,mattlant,1
76,paxdiablo,1
77,tvanfosson,1


# Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [48]:
query16= '''
WITH data AS 
(
SELECT date_trunc('month', p.creation_date)::date as month_post,
       p.id post_id,
       u.id user_id,
       date_trunc('month', u.creation_date)::date as month_user
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON u.id = p.user_id
WHERE u.id in (

SELECT p.user_id
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON u.id = p.user_id
WHERE (date_trunc('month', u.creation_date)::date = '2008-09-01') AND date_trunc('month', p.creation_date)::date = '2008-12-01'
)
)
SELECT month_post,
       count(post_id)
FROM data
GROUP BY month_post
ORDER BY month_post DESC
'''
display(pd.read_sql_query(query16, con=engine))

,month_post,count
0,2008-12-01,17641
1,2008-11-01,18294
2,2008-10-01,27171
3,2008-09-01,24870
4,2008-08-01,32


# Используя данные о постах, выведите несколько полей:
* идентификатор пользователя, который написал пост;
* дата создания поста;
* количество просмотров у текущего поста;
* сумму просмотров постов автора с накоплением.<br>
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [49]:
query17='''
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts 
ORDER BY user_id,
         creation_date
'''
display(pd.read_sql_query(query17, con=engine))

,user_id,creation_date,views_count,sum
0,1,2008-07-31 23:41:00,480476,480476
1,1,2008-07-31 23:55:38,136033,616509
2,1,2008-07-31 23:56:41,0,616509
3,1,2008-08-04 02:45:08,0,616509
4,1,2008-08-04 04:31:03,0,616509
...,...,...,...,...
243791,5696608,2008-12-23 16:00:37,0,2804
243792,5696608,2008-12-23 17:35:09,0,2804
243793,5696608,2008-12-24 01:02:48,0,2804
243794,5696608,2008-12-30 14:34:45,0,2804


# Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [50]:
query18='''
SELECT ROUND(CAST (COUNT(dt) AS NUMERIC) / count(distinct (user_id))) result

FROM (
    WITH data AS (
        SELECT DATE_TRUNC('DAY', creation_date)::DATE dt,
               user_id
        FROM stackoverflow.posts
        WHERE (DATE_TRUNC('DAY', creation_date)::DATE BETWEEN '2008-12-01' AND '2008-12-07') )

SELECT DISTINCT(dt),
       user_id
FROM data
GROUP BY user_id, dt) AS final_data
'''
display(pd.read_sql_query(query18, con=engine))

,result
0,2.0


# На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
* номер месяца;
* количество постов за месяц;
* процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.<br>
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [51]:
query19= '''
WITH data AS
(
SELECT to_char(creation_date, 'MM') creation_date,
       count(id) posts_count       
FROM stackoverflow.posts
GROUP BY to_char(creation_date, 'MM')
HAVING to_char(creation_date, 'MM') BETWEEN '09' AND '12'
)

SELECT creation_date::int, posts_count,
      ROUND(CAST((posts_count - LAG(posts_count, 1, NULL) OVER (ORDER BY creation_date)) AS numeric) / (LAG(posts_count, 1, NULL) OVER (ORDER BY creation_date)) *100, 2) percentage
FROM data
'''
display(pd.read_sql_query(query19, con=engine))

,creation_date,posts_count,percentage
0,9,70371,NaN
1,10,63102,-10.33
2,11,46975,-25.56
3,12,44592,-5.07


# Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
* номер недели;
* дата и время последнего поста, опубликованного на этой неделе.

In [52]:
query20= '''
WITH data AS(
    SELECT extract('week' from creation_date) week_creation,
           creation_date
    FROM stackoverflow.posts
    WHERE DATE_TRUNC('month', creation_date)='2008-10-01' AND user_id in (
        SELECT user_id
        FROM stackoverflow.posts
        GROUP BY user_id
        ORDER BY count(id) DESC
        LIMIT 1) )

SELECT DISTINCT
       week_creation,
       LAST_VALUE(creation_date) OVER (PARTITION BY week_creation ORDER BY week_creation)
FROM data       
ORDER BY week_creation
'''
display(pd.read_sql_query(query20, con=engine))

,week_creation,last_value
0,40.0,2008-10-05 08:16:26
1,41.0,2008-10-12 21:22:23
2,42.0,2008-10-19 06:49:30
3,43.0,2008-10-26 21:44:36
4,44.0,2008-10-31 21:54:56
